# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [21]:
#Import necessary programs
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [5]:
#Create X and Y dataframes and create training/testing data sets
X = adult_dt[adult_dt.columns.difference(['income'])]
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state in the splitting function allows us to have a reproducible output when running our model multiple times. This is useful because our model can be run multiple times and still produce identical and reliable results. 

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [7]:
#Import necessary programs
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

In [28]:
num_variables = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'] #Identify numerical values

cat_variables = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'] #Identify categorical values

#Create numerical pipeline
pipeline_num = Pipeline([
    ("imputer", KNNImputer(n_neighbors = 7, weights= "distance")),
    ("scaler", RobustScaler())
    
    ])

#Create categorical pipeline
pipeline_cat = Pipeline([
    ("imputer", SimpleImputer(strategy = 'most_frequent')),
    ("onehot", OneHotEncoder(handle_unknown= "infrequent_if_exist", drop = "first"))

])

#Create transformer
transformer = ColumnTransformer([
    ('num_transforms', pipeline_num, num_variables),
    ('cat_transforms', pipeline_cat, cat_variables)
])

In [29]:
#Display my ColumnTransformer
transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [17]:
#Import necessary classifier
from sklearn.ensemble import RandomForestClassifier

In [30]:
#Create model pipeline and add classifiier
pipeline_rfc = Pipeline([
    ("preprocessing", transformer),
    ("classifier", RandomForestClassifier())
])

In [31]:
#Dislay model pipeline
pipeline_rfc

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [ ]:
#Import cross-validation and performance metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

In [ ]:
#Define performance metrics
neg_log_loss = log_loss
roc_auc = roc_auc_score
accuracy = accuracy_score
balanced_accuracy = balanced_accuracy_score

#Apply performance metrics to scoring
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

In [ ]:
#Cross-validation on my model pipeline
cv_pipeline = cross_validate(pipeline_rfc, X_train, Y_train, cv =5, scoring = scoring, return_train_score = True)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [33]:
#Display cross validated pipeline
cv_pipeline

{'fit_time': array([10.5024929 ,  9.91184998, 10.28928804, 12.91040778,  9.70234609]),
 'score_time': array([0.29282904, 0.26003098, 0.24638271, 0.48339725, 0.2128787 ]),
 'test_neg_log_loss': array([-0.36294289, -0.36530752, -0.35633607, -0.36974686, -0.39383044]),
 'train_neg_log_loss': array([-0.08154011, -0.08140055, -0.08119754, -0.0817294 , -0.08067935]),
 'test_roc_auc': array([0.90469775, 0.90359721, 0.90451899, 0.90686805, 0.90239152]),
 'train_roc_auc': array([1., 1., 1., 1., 1.]),
 'test_accuracy': array([0.85018644, 0.84843167, 0.85563844, 0.86002633, 0.85651602]),
 'train_accuracy': array([0.99994515, 1.        , 1.        , 1.        , 1.        ]),
 'test_balanced_accuracy': array([0.7741947 , 0.76781256, 0.77764577, 0.78269562, 0.77668655]),
 'train_balanced_accuracy': array([0.9998869, 1.       , 1.       , 1.       , 1.       ])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [34]:
cv_output = pd.DataFrame(cv_pipeline).sort_values(by=['test_neg_log_loss'])

cv_output

fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4   9.702346    0.212879          -0.393830           -0.080679      0.902392   
3  12.910408    0.483397          -0.369747           -0.081729      0.906868   
1   9.911850    0.260031          -0.365308           -0.081401      0.903597   
0  10.502493    0.292829          -0.362943           -0.081540      0.904698   
2  10.289288    0.246383          -0.356336           -0.081198      0.904519   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.856516        1.000000                0.776687   
3            1.0       0.860026        1.000000                0.782696   
1            1.0       0.848432        1.000000                0.767813   
0            1.0       0.850186        0.999945                0.774195   
2            1.0       0.855638        1.000000                0.777646   

   train_balanced_accuracy  
4                 1.000000  
3                 1.000000  
1                 1.000000  
0                 0.999887  
2                 1.000000

Calculate the mean of each metric. 

In [35]:
cv_output.mean()

fit_time                   10.663277
score_time                  0.299104
test_neg_log_loss          -0.369633
train_neg_log_loss         -0.081309
test_roc_auc                0.904415
train_roc_auc               1.000000
test_accuracy               0.854160
train_accuracy              0.999989
test_balanced_accuracy      0.775807
train_balanced_accuracy     0.999977
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [37]:
#Fit pipeline to training data
pipeline_rfc.fit(X_train, Y_train)

#Create predictions based on testing data
Y_pred = pipeline_rfc.predict(X_test)
Y_pred_proba = pipeline_rfc.predict_proba(X_test)

Performance_metrics = {
    'Negative log loss' : log_loss(Y_test, Y_pred_proba), 
    'ROC AUC' : roc_auc_score(Y_test, Y_pred_proba[:, 1]), 
    'Accuracy' : accuracy_score(Y_test, Y_pred),
    'Balanced accuracy' : balanced_accuracy_score(Y_test, Y_pred)
}

In [ ]:
#Display dictionary
Performance_metrics

{'Negative log loss': 0.387647283794311,
 'ROC AUC': 0.8996518436461465,
 'Accuracy': 0.8540280479066434,
 'Balanced accuracy': 0.7750214337016046}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding the income variable when loading our data was convinent as it reduces extra steps when creating our model in two ways. One, we don't need to recode the income column when creating our dataframes. Two, it allows us to easily identify records with incomes above $50K. 

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.